On cherche dans ce notebook à comparer l'efficacité des différents taux de contamination pour l'algorithme isolation forest.

Trois méthodes ont été utilisées :
- auto : on laisse l'algorithme décider du taux
- fixe : on fixe le taux à 15%
- var : on fixe un taux différent pour chaque cluster, égal au pourcentage de messages détectés comme non haineux par le modèle de validation (en espérant que l'algorithme se débarasse de ces messages)

### Imports et initialisations

In [1]:
from clustertools import *

In [2]:
clusters_dir = "clusters/no_outliers_bert_som25/clean_bert_som25/"

### Charger les clusters en mémoire

In [3]:
dir_it = os.scandir(path=clusters_dir)

In [4]:
names = [entry.name for entry in dir_it]

In [5]:
def extr_num(string):
    num = re.search("([0-9]+)", string)
    return int(num.group(1)) if num else -1

In [6]:
names.sort(key=extr_num)

In [7]:
names

['auto_clean_0',
 'fixe_clean_0',
 'var_clean_0',
 'auto_clean_12',
 'fixe_clean_12',
 'var_clean_12',
 'auto_clean_13',
 'fixe_clean_13',
 'var_clean_13',
 'auto_clean_14',
 'fixe_clean_14',
 'var_clean_14',
 'auto_clean_17',
 'fixe_clean_17',
 'var_clean_17',
 'auto_clean_19',
 'fixe_clean_19',
 'var_clean_19']

In [8]:
auto_rc = [load_cluster(clusters_dir + name) for name in names if 'auto' in name]
var_rc = [load_cluster(clusters_dir + name) for name in names if 'var' in name]
fixe_rc = [load_cluster(clusters_dir + name) for name in names if 'fixe' in name]
auto_cc = [[m.split() for m in c] for c in auto_rc]
var_cc = [[m.split() for m in c] for c in var_rc]
fixe_cc = [[m.split() for m in c] for c in fixe_rc]

### Chargement du modèle de validation

In [9]:
import torch 
from transformers import CamembertForSequenceClassification, CamembertTokenizer

tokenizer = CamembertTokenizer.from_pretrained("models/tweetbert", do_lowercase=False)
classifier = CamembertForSequenceClassification.from_pretrained("models/tweetbert_FT_rand_balanced")

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
r = classifier.to(device)

#### Prédiction du pourcentage de messages haineux

In [11]:
def predict(sample):
    pred = []
    for n in range(0, len(sample), 10):
        phrases = sample[n:n+10]
        enc = tokenizer(phrases, padding=True, truncation=True, max_length=512, return_tensors='pt')
        enc.to('cuda:0')
        with torch.no_grad():
            outp = classifier(**enc)
        p = torch.nn.functional.softmax(outp.logits, dim=1)
        labels = torch.argmax(p, dim=1)
        pred.append(labels)
    return torch.cat(pred)

In [12]:
%%time 
auto_pred = []
for clust in auto_rc:
    auto_pred.append(predict(clust))
    
fixe_pred = []
for clust in fixe_rc:
    fixe_pred.append(predict(clust))

var_pred = []
for clust in var_rc:
    var_pred.append(predict(clust))

CPU times: user 15.9 s, sys: 0 ns, total: 15.9 s
Wall time: 15.9 s


In [13]:
def hate_perc(hate_pred):
    hper = []
    for i, p in enumerate(hate_pred):
        bc = torch.bincount(p, minlength=2)
        perc = (bc[1] / (bc[0]+bc[1])).item() * 100
        hper.append(perc)
    return hper

In [14]:
auto_per = hate_perc(auto_pred)
fixe_per = hate_perc(fixe_pred)
var_per = hate_perc(var_pred)

Pourcentage et nombre de messages pour les clusters avant passage d'isolation forest

In [15]:
no_IF_hper = [19.11, 11.27, 14.29, 83.51, 1.37, 98.31]
no_IF_n_msg = [225, 142, 133, 97, 73, 354]

In [16]:
def print_res(hper):
    s = 0
    sp = 0
    print("% sans\t| % avec| diff\t| diff pondérée par nb msg\n---------------------------------")
    for nh, nm, ih in zip(no_IF_hper, no_IF_n_msg, hper):#, n_msg):
        diff = nh-ih if nh < 50 else ih-nh
        diffp = diff*nm
        print("{}\t| {:.2f}\t| {:.2f}\t| {:.2f}".format(nh, ih, diff, diffp))
        s += diff
        sp += diffp
    print("\ntotal diff : {:.2f}\ttotal diff pondérée par nb msg : {:.2f}\n".format(s, sp))    

#### On compare les résultats de chaque méthode aux clusters de base
- pour chaque cluster on calcule le pourcentage de messages détectés comme haineux par le modèle de validation
- on compare avec le pourcentage des clusters de base
- si le cluster de base a moins de 20% de messages haineux, on cherche à faire diminuer ce pourcentage avec IF, la différence absolue entre les deux sera donc comptabilisée comme positive si le pourcentage post IF est inférieur à celui du cluster de base et négative sinon
- inversement si le cluster de base a plus de 80% de messages haineux, on cherche à faire augmenter ce pourcentage avec IF, la différence absolue entre les deux sera donc comptabilisée comme négative si le pourcentage post IF est inférieur à celui du cluster de base et positive sinon

In [17]:
print("Auto :")
print_res(auto_per)
print("Fixe :")
print_res(fixe_per)
print("Var :")
print_res(var_per)

Auto :
% sans	| % avec| diff	| diff pondérée par nb msg
---------------------------------
19.11	| 18.75	| 0.36	| 81.00
11.27	| 12.32	| -1.05	| -148.94
14.29	| 11.63	| 2.66	| 354.06
83.51	| 86.32	| 2.81	| 272.16
1.37	| 2.94	| -1.57	| -114.70
98.31	| 98.57	| 0.26	| 92.54

total diff : 3.47	total diff pondérée par nb msg : 536.13

Fixe :
% sans	| % avec| diff	| diff pondérée par nb msg
---------------------------------
19.11	| 18.85	| 0.26	| 58.91
11.27	| 12.50	| -1.23	| -174.66
14.29	| 11.50	| 2.79	| 370.48
83.51	| 87.80	| 4.29	| 416.60
1.37	| 3.23	| -1.86	| -135.47
98.31	| 99.00	| 0.69	| 245.44

total diff : 4.95	total diff pondérée par nb msg : 781.30

Var :
% sans	| % avec| diff	| diff pondérée par nb msg
---------------------------------
19.11	| 19.23	| -0.12	| -27.17
11.27	| 11.90	| -0.63	| -90.14
14.29	| 11.40	| 2.89	| 383.90
83.51	| 87.65	| 4.14	| 402.00
1.37	| 2.78	| -1.41	| -102.77
98.31	| 98.56	| 0.25	| 89.64

total diff : 5.12	total diff pondérée par nb msg : 655.46



Toutes les méthodes améliorent la pertinence des cluster et une contamination fixe à 15% donne les meilleurs résultats